In [1]:
# Problem 1: Uncomputing
import cirq
import numpy as np

L=4
qubits=cirq.LineQubit.range(L)

#I'm going to define a ZZZ rotation since that will be more useful for problem 2, to get the XXX operation just add hadamards

def ZZZrotation(i,j,k,A,theta): # constructs the circuit for the uncomputing implementation of e^{i theta Z_i Z_j Z_k}
    res = [] # container to store gates -- note, this assumes ancilla A is in 0
    res.append(cirq.CNOT(qubits[i],qubits[A]))
    res.append(cirq.CNOT(qubits[j],qubits[A]))
    res.append(cirq.CNOT(qubits[k],qubits[A]))
    # now do the Z rotation
    res.append(cirq.rz(2*theta).on(qubits[A]))
    # undo the operations
    res.append(cirq.CNOT(qubits[k],qubits[A]))
    res.append(cirq.CNOT(qubits[j],qubits[A]))
    res.append(cirq.CNOT(qubits[i],qubits[A]))
    return res # returns the gate

test = cirq.Circuit(ZZZrotation(0,1,2,3,np.pi/4))
print(test)

def Hlayer(i,j,k):
    res = []
    res.append(cirq.H(qubits[i]))
    res.append(cirq.H(qubits[j]))
    res.append(cirq.H(qubits[k]))
    return res

# make a function for the XXX rotation
def XXXrotation(i,j,k,A,theta):
    res=[]
    res.append(Hlayer(i,j,k))
    res.append(ZZZrotation(i,j,k,A,theta))
    res.append(Hlayer(i,j,k))
    return res

test = cirq.Circuit(XXXrotation(0,1,2,3,np.pi/4))
print(test)

# now verify that it works:

s=cirq.Simulator()

test = cirq.Circuit(XXXrotation(0,1,2,3,np.pi/8))
print(s.simulate(test))

test = cirq.Circuit(XXXrotation(0,1,2,3,np.pi/4))
print(s.simulate(test))

test = cirq.Circuit(XXXrotation(0,1,2,3,np.pi/2))
print(s.simulate(test))

test = cirq.Circuit(XXXrotation(0,1,2,3,5*np.pi/4))
print(s.simulate(test))

test = cirq.Circuit(XXXrotation(0,1,2,3,3*np.pi/2))
print(s.simulate(test))

test = cirq.Circuit(XXXrotation(0,1,2,3,np.pi))
print(s.simulate(test))

# success!
print("Success!")
    

0: ───@──────────────────────────────@───
      │                              │
1: ───┼───@──────────────────────@───┼───
      │   │                      │   │
2: ───┼───┼───@──────────────@───┼───┼───
      │   │   │              │   │   │
3: ───X───X───X───Rz(0.5π)───X───X───X───
                                     ┌──┐   ┌──┐
0: ───H───@──────────────────────────────────@─────H───
          │                                  │
1: ───H───┼───@───────────────────────@──────┼H────────
          │   │                       │      │
2: ───H───┼───┼───@──────────────@────┼H─────┼─────────
          │   │   │              │    │      │
3: ───────X───X───X───Rz(0.5π)───X────X──────X─────────
                                     └──┘   └──┘
measurements: (no measurements)
output vector: 0.924|0000⟩ - 0.383j|1110⟩
measurements: (no measurements)
output vector: 0.707|0000⟩ - 0.707j|1110⟩
measurements: (no measurements)
output vector: -1j|1110⟩
measurements: (no measurements)
output vector: 

In [23]:
# Problem 2: QAOA

import cirq
# import qsimcirq
import numpy as np
import math
import random

L=7
qubits=cirq.LineQubit.range(L)

# first, define a function to make a random triple (all three numbers must be distinct)

#VERIFIED
def random_triple(x):
    res = []
    rv = math.floor(random.uniform(0,x))
    res.append(rv)
    while(len(res)<3):
        rv = math.floor(random.uniform(0,x))
        if(not (rv in res)):
            res.append(rv)
    res.sort()
    return res

print("test triplet", random_triple(L-1))

#VERIFIED
def makeproblem(numcons,x): # make the problem, ensuring no duplications
    res = []
    res.append(random_triple(x))
    while(len(res)<numcons):
        randcons=random_triple(x)
        if(not (randcons in res)):
            res.append(randcons)
            
    return res


graph = makeproblem(12,L-1)
    
print("Hp",graph)

Vlist = [] # now generate coefficients
for j in range(12):
    Vlist.append((-1)**(math.floor(random.uniform(0,2))))#Possibly the least efficient way to choose -1 or 1

print("Triplet Values",Vlist)

# we also need a way to calculate the energy of a given bitstring (binary index state)

binaryformat = '{0:0' + str(len(qubits)) + 'b}'

#CLASSICAL GROUND ENERGY - VERIFIED
def get_energy(state, conslist, vlist):
    en = 0
    bitstring=binaryformat.format(state)
    for j in range(len(conslist)):
        en += vlist[j]*(-1)**(int(bitstring[conslist[j][0]])+ int(bitstring[conslist[j][1]]) +int(bitstring[conslist[j][2]]))
    return en
# now test getting the energy by simply enumerating all the states (straightforward for this small L)

enlist = []
minE = 0
for j in range(2**(L-1)):
    Enow =get_energy(j,graph,Vlist)
    enlist.append(Enow)
    if(Enow<minE):
        minE = Enow

print("Ground State Energy:",minE)

# worth finding the position(s) of the minimum energy as well (it is probably not unique), since we may be intersted in finding the probability of obtaining that as well
#VERIFIED
def searchlist(thelist,val):
    positions = []
    for j in range(len(thelist)):
        if(thelist[j]==val):
            positions.append(j)
    return positions

GSposlist=searchlist(enlist,minE)

print("Ground states(decimal)", GSposlist)
# ok good, now we've made the problem -- time to construct the circuit

#alpha beta, triplets, values, number of qubits (for ancilla qubit-always last)
#BUILD THE CIRCUIT
#VERIFIED
def QAOA_circuit2(alpha, beta, conslist, vlist, x): #inputs are angles alpha and beta, list of triples conslist, list of coefficients vlist, number of qubits x
    c=cirq.Circuit()
    for j in range(L-1): # start by hadamarding all qubits
        c.append(cirq.H(qubits[j]))
    for j in range(len(conslist)): # now apply the problem Hamiltonian
        c.append(ZZZrotation(conslist[j][0],conslist[j][1],conslist[j][2],x-1,alpha*vlist[j]))
    for j in range(L-1):
        c.append(cirq.rx(-beta).on(qubits[j])) #want this angle to be negative as an applied rotation
        
    c.append(cirq.measure(*qubits,key='z'))
    return c

# time to run the circuit: -- choose a simulator

s = cirq.Simulator()
# s = qsimcirq.QSimSimulator()
counts = 5000 # 5000 counts is probably overkill for a problem this small

# we need a function to process the data, using the method sketched in the textbook
#WHAT IS THE AVERAGE TIMES THE GROUNDSTATE AND ENERGY ARE HIT - PROBABILITY
def getvals(data,GSpos):
    enavg = 0
    gsavg = 0
    for key, value in data.items():
        #bitstring = binaryformat.format(int(key))# if we needed to generate the bitstring here, but we already made the energy list above
        Enow = get_energy(int(key),graph,Vlist)
        enavg += int(value) * Enow
        if(Enow == minE):
            gsavg += int(value) #add the number of counts that found a ground state
    enavg = enavg/counts #divide by counts for proper averagin
    gsavg = gsavg/counts
    return [enavg,gsavg]

# time to test it

#print(QAOA_circuit2(0.4,0.7,graph,Vlist,L))

print("\n Testing this for a single pair of angles: \n")

testsamples = s.run(QAOA_circuit2(0.4,0.7,graph,Vlist,L),repetitions=counts)
testdata = testsamples.histogram(key='z')
testvals = getvals(testdata,GSposlist)
print("P(ground_energy), P(ground_state)", testvals)





test triplet [0, 2, 3]
Hp [[1, 2, 3], [1, 2, 4], [2, 4, 5], [0, 2, 5], [0, 3, 5], [2, 3, 5], [0, 1, 3], [0, 4, 5], [1, 3, 4], [0, 1, 2], [1, 3, 5], [3, 4, 5]]
Triplet Values [-1, 1, 1, 1, -1, -1, -1, -1, -1, -1, 1, 1]
Ground State Energy: -6
Ground states(decimal) [42, 43]

 Testing this for a single pair of angles: 

P(ground_energy), P(ground_state) [-1.6644, 0.1102]


In [22]:
print("\n Now survey angles: \n")

bestEQAOA=0 # track best energy
bestGSQAOA=0 # separately, track best probability of finding a ground state
datalist = []
for j in range(10):
    for k in range(10):
        samp=s.run(QAOA_circuit2(j*np.pi/40,k*np.pi/40,graph,Vlist,L),repetitions=counts)
        data = samp.histogram(key='z')
        vals=getvals(data,GSposlist)
        datalist.append([j*np.pi/40,k*np.pi/40,vals[0],vals[1]])
        if(vals[0]<bestEQAOA):
            bestEQAOA = vals[0]
            bestdataE = [j*np.pi/40,k*np.pi/40,vals[0]/minE,vals[1]]
        if(vals[1]>bestGSQAOA):
            bestGSQAOA = vals[1]
            bestdataGS = [j*np.pi/40,k*np.pi/40,vals[0]/minE,vals[1]]
        print(vals[0])

# print(datalist)
print("\n Best results, for energy and ground state (energy divided by min energy to track approximation ratio):")
print(bestdataE)
print(bestdataGS)


 Now survey angles: 

0.0696
0.0304
0.1036
-0.004
0.0784
0.006
-0.04
-0.0004
0.0368
-0.0752
0.0288
-0.3828
-0.806
-1.1936
-1.3692
-1.6472
-1.6408
-1.702
-1.7444
-1.5272
0.0252
-0.6388
-1.2892
-1.8444
-2.2708
-2.64
-2.6952
-2.8
-2.7688
-2.4036
0.068
-0.6744
-1.2112
-1.7948
-2.362
-2.716
-2.9072
-2.988
-2.7692
-2.6228
-0.0044
-0.4944
-0.9544
-1.3728
-1.6812
-2.0312
-2.146
-2.2116
-2.2816
-2.1516
0.058
-0.2112
-0.5548
-0.7668
-0.9796
-1.1476
-1.2752
-1.3016
-1.2596
-1.2528
-0.0604
-0.1024
-0.2392
-0.3068
-0.5004
-0.5416
-0.5688
-0.6344
-0.6892
-0.6804
-0.0424
-0.1472
-0.1732
-0.0736
-0.3316
-0.3248
-0.3824
-0.4316
-0.3688
-0.4924
-0.0636
-0.0424
-0.1356
-0.21
-0.34
-0.3264
-0.3592
-0.4828
-0.5676
-0.58
0.0236
-0.0704
-0.188
-0.3404
-0.3592
-0.426
-0.5284
-0.57
-0.54
-0.6088

 Best results, for energy and ground state (energy divided by min energy to track approximation ratio):
[0.23561944901923448, 0.5497787143782138, 0.2988, 0.0828]
[0.23561944901923448, 0.7068583470577035, 0.2622799999

In [7]:
print(np.array(datalist))

[[ 0.00000000e+00  0.00000000e+00  4.88000000e-02  1.34000000e-02]
 [ 0.00000000e+00  7.85398163e-02  8.56000000e-02  1.30000000e-02]
 [ 0.00000000e+00  1.57079633e-01  2.68000000e-02  1.72000000e-02]
 [ 0.00000000e+00  2.35619449e-01 -3.68000000e-02  1.62000000e-02]
 [ 0.00000000e+00  3.14159265e-01  3.28000000e-02  1.66000000e-02]
 [ 0.00000000e+00  3.92699082e-01  1.64000000e-02  1.32000000e-02]
 [ 0.00000000e+00  4.71238898e-01 -1.04000000e-02  1.36000000e-02]
 [ 0.00000000e+00  5.49778714e-01  8.44000000e-02  1.48000000e-02]
 [ 0.00000000e+00  6.28318531e-01 -2.52000000e-02  1.80000000e-02]
 [ 0.00000000e+00  7.06858347e-01 -1.88000000e-02  1.42000000e-02]
 [ 7.85398163e-02  0.00000000e+00  1.72000000e-02  1.84000000e-02]
 [ 7.85398163e-02  7.85398163e-02 -4.18800000e-01  1.94000000e-02]
 [ 7.85398163e-02  1.57079633e-01 -7.70000000e-01  2.02000000e-02]
 [ 7.85398163e-02  2.35619449e-01 -1.15560000e+00  3.34000000e-02]
 [ 7.85398163e-02  3.14159265e-01 -1.44920000e+00  3.36000000e

In [3]:
print("\n Bonus: two layers. \n Now survey angles for two layers (6 values each)")

# for your reference, here's a version that takes arbitrary lists of angles alphalist and betalist (note the lists need to be the same length), to do many layers

def QAOA_circuitP(alphalist, betalist, conslist, vlist, x):
    c=cirq.Circuit()
    for j in range(L-1): # start by hadamarding all qubits
        c.append(cirq.H(qubits[j]))
    for k in range(len(alphalist)):
        for j in range(len(conslist)): # now apply the problem Hamiltonian
            c.append(ZZZrotation(conslist[j][0],conslist[j][1],conslist[j][2],x-1,alphalist[k]*vlist[j]))
        for j in range(L-1):
            c.append(cirq.rx(-betalist[k]).on(qubits[j])) #again want these to be negative as applied rotations
    c.append(cirq.measure(*qubits,key='z'))
    return c

bestEQAOA2=0 # track best energy
bestGSQAOA2=0 # separately, track best probability of finding a ground state
for j in range(6):
    for k in range(6):
        for l in range(6):
            for m in range(6):
                alist = [j*np.pi/30,k*np.pi/30]
                blist = [l*np.pi/30,m*np.pi/30]
                samp=s.run(QAOA_circuitP(alist,blist,graph,Vlist,L),repetitions=counts)
                data = samp.histogram(key='z')
                vals=getvals(data,GSposlist)
                if(vals[0]<bestEQAOA2):
                    bestEQAOA2 = vals[0]
                    bestdataE2 = [alist,blist,vals[0]/minE,vals[1]]
                if(vals[1]>bestGSQAOA2):
                    bestGSQAOA2 = vals[1]
                    bestdataGS2 = [alist,blist,vals[0]/minE,vals[1]]
                    
                    
print("\n Best results, for two layers, for energy and ground state (energy divided by min energy to track approximation ratio):")
print(bestdataE2)
print(bestdataGS2)


 Bonus: two layers. 
 Now survey angles for two layers (6 values each)

 Best results, for two layers, for energy and ground state (energy divided by min energy to track approximation ratio):
[[0.10471975511965977, 0.20943951023931953], [0.5235987755982988, 0.41887902047863906], 0.61, 0.2072]
[[0.20943951023931953, 0.3141592653589793], [0.5235987755982988, 0.5235987755982988], 0.5018666666666667, 0.2662]


In [4]:
# Problem 3: now include errors

# define a routine to insert a fixed number of random errors

print("\n Problem 3: simulation with errors:")

def err_circuit(circuit,depth,num_errs,x): # adds a random error to a circuit
    mycircuit = circuit
    for j in range(num_errs):
        rpos = math.floor(random.uniform(0,depth)) # random time
        rv = random.uniform(0,1)
        rq = math.floor(random.uniform(0,x)) # random qubit
        if(rv<(1/3)):
            gate= cirq.X(qubits[rq])
        elif(rv<(2/3)):
            gate= cirq.Y(qubits[rq])
        else:
            gate= cirq.Z(qubits[rq])
        mycircuit.insert(rpos,gate) #insert the random error
        
    return mycircuit # return the circuit with an error inserted

def err_avg(circuit,depth,num_errs,x,probgraph,vlist,L,trials):
    avgE = 0
    avgGS = 0
    for j in range(trials):
        samp=s.run(err_circuit(circuit,depth,num_errs,x),repetitions=counts)
        data = samp.histogram(key='z')
        vals=getvals(data,GSposlist)
        avgE+=vals[0]
        avgGS+=vals[1]
    return [avgE/trials,avgGS/trials]

trials = 100
# use the best results from earlier, and don't trigger errors in the ancilla (they will be unrepresentatively destructive)
QAOA2 = QAOA_circuit2(bestdataE[0],bestdataE[1],graph,Vlist,L)
QAOA4 = QAOA_circuitP(bestdataE2[0],bestdataE2[1],graph,Vlist,L)

print("\n 1 error circuit average, depth 2 \n")
oneerr2 = err_avg(QAOA2,len(QAOA2),1,L-1,graph,Vlist,L,trials)
print(oneerr2)
print("\n 1 error circuit average, depth 4 \n")
oneerr4 = err_avg(QAOA4,len(QAOA4),1,L-1,graph,Vlist,L,trials)
print(oneerr4)
print("\n 2 error circuit average, depth 4 \n")
twoerr4 = err_avg(QAOA4,len(QAOA4),2,L-1,graph,Vlist,L,trials)
print(twoerr4)


 Problem 3: simulation with errors:

 1 error circuit average, depth 2 

[-0.022011999999999962, 0.049525999999999994]

 1 error circuit average, depth 4 

[-0.03722, 0.04514800000000002]

 2 error circuit average, depth 4 

[-0.04709599999999999, 0.049524000000000006]
